In [1]:
import sympy as sp
import numpy as np
import itertools as it
import functools as ft
import pickle
import multiprocessing as mp
mp.set_start_method('fork') # 'spawn' might be better on some platforms

In [2]:
the_graph = graphs.McLaughlinGraph()
num_vert = the_graph.order()

In [3]:
def coeff_func_for_part(box_size,partition,ext_ext_edges=[],ext_int_edges=[],int_int_edges=[],num_int_vert=0):
    """Constructs a coefficient function for a given partition
    
    Constructs a function that returns the coefficient of tuples in the `box_size`-th box space
    of the spin planar algebra, specified by the partition.
    
    Parameters
    ----------
    box_size : int
    partition : list of sets of integers 0 .. box_size-1. You can omit singleton sets, so that
    [{1,2}] and [{0},{1,2}] would have the same effect.
    ext_ext_edges : list of pairs (a, b)
    """

    def coeff_func(*variables):
        if len(variables) == 1 and variables[0] == "diag":
            return diag_for_4_box_elem(partition,ext_ext_edges,ext_int_edges,int_int_edges,num_int_vert)
        if len(variables) != box_size:
            raise Exception("number of variables does not match the box size.")
        for part in partition:
            vars_in_part = [variables[i] for i in range(box_size) if i in part]
            if len(set(vars_in_part)) > 1:
                return 0
        if any(True for edge in ext_ext_edges if not variables[edge[0]] in the_graph.neighbors(variables[edge[1]])):
            return 0
        if num_int_vert == 0:
            return 1
        int_vert_candidates = []
        for i in range(num_int_vert):
            ext_vert_neighs = [set(the_graph.neighbors(variables[edge[0]])) for edge in ext_int_edges if edge[1] == i]
            cand_set = ft.reduce(lambda x, y: x.intersection(y), ext_vert_neighs, set(range(num_vert)))
            int_vert_candidates.append(cand_set)
        def branch_candidates_for(prev_verts):
            i = len(prev_verts)
            prev_int_verts_conn_by_edges_0 = {edge[0] for edge in int_int_edges if edge[1] == i and edge[0] < i}
            prev_int_verts_conn_by_edges_1 = {edge[1] for edge in int_int_edges if edge[0] == i and edge[1] < i}
            prev_int_verts_conn_by_edges = prev_int_verts_conn_by_edges_0.union(prev_int_verts_conn_by_edges_1)
            nvc = [v for v in int_vert_candidates[i] if not any(True for j in prev_int_verts_conn_by_edges if not v in the_graph.neighbors(prev_verts[j]))]
            if len(prev_verts) == num_int_vert - 1:
                return [prev_verts + [v] for v in nvc]
            else:
                return [x for v in nvc for x in branch_candidates_for(prev_verts + [v])]
        valid_int_conf = branch_candidates_for([])
        return len(valid_int_conf)
    return coeff_func

In [4]:
def paths_for_4_box_elem(partition,ext_ext_edges=[],ext_int_edges=[],int_int_edges=[],num_int_vert=0):
    output =""
    if num_int_vert == 1:
        output += r'\node (inn_v0) at (0,0) [inn-vert] {};' + '\n'
    elif num_int_vert == 2:
        output += r'\node (inn_v0) at (-0.3,0) [inn-vert] {};' + '\n'
        output += r'\node (inn_v1) at (0.3,0) [inn-vert] {};' + '\n'
    elif num_int_vert == 3:
        output += r'\node (inn_v0) at (-0.4,0.3) [inn-vert] {};' + '\n'
        output += r'\node (inn_v1) at (0.4,0.3) [inn-vert] {};' + '\n'
        output += r'\node (inn_v2) at (0,-0.4) [inn-vert] {};' + '\n'
    elif num_int_vert == 4:
        output += r'\node (inn_v0) at (0,0.5) [inn-vert] {};' + '\n'
        output += r'\node (inn_v1) at (0.5,0) [inn-vert] {};' + '\n'
        output += r'\node (inn_v2) at (0,-0.5) [inn-vert] {};' + '\n'
        output += r'\node (inn_v3) at (-0.5,0) [inn-vert] {};' + '\n'
    if any({0,1}.issubset(s) for s in partition):
        output += r'\draw [thick,double] (corner0) -- (corner1);' + '\n'
    if any({1,2}.issubset(s) for s in partition):
        output += r'\draw [thick,double] (corner1) -- (corner2);' + '\n'
    if any({2,3}.issubset(s) for s in partition):
        output += r'\draw [thick,double] (corner2) -- (corner3);' + '\n'
    if any({3,0}.issubset(s) for s in partition):
        output += r'\draw [thick,double] (corner3) -- (corner0);' + '\n'
    if not any({0,1}.issubset(s) for s in partition) and not any({3,0}.issubset(s) for s in partition) and any({0,2}.issubset(s) for s in partition):
        output += r'\draw [thick,double] (corner0) -- (corner2);' + '\n'    
    if not any({0,1}.issubset(s) for s in partition) and not any({1,2}.issubset(s) for s in partition) and any({1,3}.issubset(s) for s in partition):
        output += r'\draw [thick,double] (corner1) -- (corner3);' + '\n'    
    for ee_edge in ext_ext_edges:
        output += r'\draw [] (corner%d) -- (corner%d);' % ee_edge + '\n'
    for ei_edge in ext_int_edges:
        output += r'\draw [] (corner%d) -- (inn_v%d);' % ei_edge + '\n'
    for ii_edge in int_int_edges:
        output += r'\draw [] (inn_v%d) -- (inn_v%d);' % ii_edge + '\n'
    return output

def diag_for_4_box_elem(partition,ext_ext_edges=[],ext_int_edges=[],int_int_edges=[],num_int_vert=0):
    output = r'\begin{tikzpicture}' + '\n'
    output += r'\draw[bgrect] (-1.3,-1.3) rectangle (1.3,1.3);' + '\n'
    output += r'\node (corner0) at (-1,1) [ext-vert] {};' + '\n'
    output += r'\node (corner1) at (1,1) [ext-vert] {};' + '\n'
    output += r'\node (corner2) at (1,-1) [ext-vert] {};' + '\n'
    output += r'\node (corner3) at (-1,-1) [ext-vert] {};' + '\n'
    output += paths_for_4_box_elem(partition,ext_ext_edges,ext_int_edges,int_int_edges,num_int_vert)
    output += r'\end{tikzpicture}' + '\n'
    return output

helper functions to get functions by rotated patterns

In [5]:
def rot_ind(x,step,length):
    y = x + step
    if y < length:
        return y
    return y-length

def rot_partition(partition,step,length):
    """Rotate partitions
    
    Parameters
    ==========
    partition : collection of sets of indexes for external leaves, singletons can be omitted
    step : int, for step of rotations
    length : int, period of rotation
    """
    return [set([rot_ind(x,step,length) for x in block]) for block in partition]

def rot_e_e_edges(edges,step,length):
    """Rotated edges between external leaves
    
    Parameters
    ==========
    edges : collection of pairs of indexes for external leaves
    step : int, for step of rotations
    length : int, period of rotation
    """
    return [(rot_ind(edge[0],step,length),rot_ind(edge[1],step,length)) for edge in edges]

def rot_e_i_edges(edges,step,length):
    """Rotated edges between external leaves and internal vertices
    
    Parameters
    ==========
    edges : collection of pairs (index for external leaves, index for interval vertices)
    step : int, for step of rotations
    length : int, period of rotation
    """
    return [(rot_ind(edge[0],step,length),edge[1]) for edge in edges]

def coeff_funcs_parts_e_e_rotated_3(partition,ee_edges):
    """Rotated coefficient functions based on edges between external leaves
    
    Parameters
    ==========
    partition : collection of sets of indexes for external leaves, singletons can be omitted
    ee_edges : collection of pairs of indexes for external leaves
    """
    return [coeff_func_for_part(3,rot_partition(partition,i,3),rot_e_e_edges(ee_edges,i,3)) for i in range(3)]
def coeff_funcs_parts_e_e_e_i_rotated_3(partition,ee_edges,ei_edges,num_int_e):
    """Rotated coefficient functions based on edges between external leaves,
    and those between external leaves and internal vertices
    
    Parameters
    ==========
    partition : collection of sets of indexes for external leaves, singletons can be omitted
    ee_edges : collection of pairs of indexes for external leaves
    ei_edges : collection of pairs (index for external leaves, index for interval vertices)
    num_int_e : int, number of internal vertices
    """
    return [coeff_func_for_part(3,rot_partition(partition,i,3),rot_e_e_edges(ee_edges,i,3),rot_e_i_edges(ei_edges,i,3),[],num_int_e) for i in range(3)]

In [6]:
nc_partitions_3 = [[],[{0,1}],[{1,2}],[{0,2}],[{0,1,2}]]

In [7]:
func_list_nc_3 = [coeff_func_for_part(3,this_nc_part) for this_nc_part in nc_partitions_3]
func_list_6_8 = coeff_funcs_parts_e_e_rotated_3([],[(0,2)])
func_list_9_11 = coeff_funcs_parts_e_e_rotated_3([{0,2}],[(0,1)])
func_list_12_14 = coeff_funcs_parts_e_e_rotated_3([],[(0,1),(1,2)])
func_list_15_16 = [coeff_func_for_part(3,[],[],[(0,0),(1,0),(2,0)],[],1),coeff_func_for_part(3,[],[(0,1),(1,2),(0,2)])]

In [8]:
A_funcs = func_list_nc_3 + func_list_6_8 + func_list_9_11 + func_list_12_14 + [func_list_15_16[1]]

get representatives of orbits for triples of vertices

In [9]:
trip_vert_orb_rep = [(0,0,0)] # start with diagonal
# 0 - neighbor - 0
neigh_0 = the_graph.neighbors(0)[0]
trip_vert_orb_rep += [(0, 0, neigh_0), (0, neigh_0, 0), (neigh_0, 0, 0)]
# 0 - neghbor - vertex at distance 2 from 0, distance 1 from neighbor
vert_set_dist_1 = set(the_graph.neighbors(0))
neighbors_neigh_0 = set(the_graph.neighbors(neigh_0))
vert_set_dist_2 = neighbors_neigh_0.difference(vert_set_dist_1).difference({0})
dist_2_fr_0 = list(vert_set_dist_2)[0]
trip_vert_orb_rep += [(0, neigh_0, dist_2_fr_0), (dist_2_fr_0, 0, neigh_0), (neigh_0, dist_2_fr_0, 0)]
# 0 - 0 - vertex at distance 2 from 0
trip_vert_orb_rep += [(0, 0, dist_2_fr_0), (0, dist_2_fr_0, 0), (dist_2_fr_0, 0, 0)]
# 0 - neighbor - vertex at distance 2 from both
unconn_vert_set = set(the_graph.vertices()).difference(vert_set_dist_1).difference(neighbors_neigh_0).difference({0, neigh_0})
unconn_vert = list(unconn_vert_set)[0]
trip_vert_orb_rep += [(0, neigh_0, unconn_vert), (0, unconn_vert, neigh_0), (unconn_vert, 0, neigh_0)]
# unconnected triple
unconn_verts_from_0_and_dist_2_fro_0 = set(the_graph.vertices()).difference(vert_set_dist_1).difference(set(the_graph.neighbors(dist_2_fr_0))).difference({0, dist_2_fr_0})
unconn_from_both = list(unconn_verts_from_0_and_dist_2_fro_0)[0]
trip_vert_orb_rep.append((0, dist_2_fr_0, unconn_from_both))
# triangle
triang_vert_set = neighbors_neigh_0.intersection(vert_set_dist_1)
if len(triang_vert_set) > 0 :
    trip_vert_orb_rep.append((0, neigh_0, list(triang_vert_set)[0]))
print(len(trip_vert_orb_rep))
trip_vert_orb_rep

15


[(0, 0, 0),
 (0, 0, 48),
 (0, 48, 0),
 (48, 0, 0),
 (0, 48, 257),
 (257, 0, 48),
 (48, 257, 0),
 (0, 0, 257),
 (0, 257, 0),
 (257, 0, 0),
 (0, 48, 256),
 (0, 256, 48),
 (256, 0, 48),
 (0, 257, 1),
 (0, 48, 128)]

check the relation in 3-box space; the 16th element in Liu's presentation should be in the span of the first 14 for this graph

In [10]:
def myfunc1(x):
    return [func(x[0],x[1],x[2]) for func in A_funcs]
def myfunc2(x):
    return func_list_15_16[0](x[0],x[1],x[2])

if __name__ == '__main__':
    with mp.Pool() as p:
        A_array = p.map(myfunc1, trip_vert_orb_rep)
        b_array = p.map(myfunc2, trip_vert_orb_rep)

In [11]:
sols = sp.linsolve((sp.Matrix(A_array),sp.Matrix(b_array))); sols

{(32, 24, 24, 24, 8, -12, -12, -12, -4, -4, -4, 2, 2, 2, 0)}

dimension of the 3-box space

In [12]:
Apr_funcs = func_list_nc_3 + func_list_6_8 + func_list_9_11 + func_list_12_14 + func_list_15_16
Apr_array = [[func(x,y,z) for func in Apr_funcs] for x, y, z in trip_vert_orb_rep]
sp.Matrix(Apr_array).rank()

15

helper functions to get rotated elements in the 4-box space

In [13]:
def coeff_funcs_parts_e_e_rotated_4(partition,ee_edges):
    return [coeff_func_for_part(4,rot_partition(partition,i,4),rot_e_e_edges(ee_edges,i,4)) for i in range(4)]
def coeff_funcs_parts_e_e_e_i_rotated_4(partition,ee_edges,ei_edges,num_int_e):
    return [coeff_func_for_part(4,rot_partition(partition,i,4),rot_e_e_edges(ee_edges,i,4),rot_e_i_edges(ei_edges,i,4),[],num_int_e) for i in range(4)]
def coeff_funcs_parts_e_e_e_i_i_i_rotated_4(partition,ee_edges,ei_edges,ii_edges,num_int_e):
    return [coeff_func_for_part(4,rot_partition(partition,i,4),rot_e_e_edges(ee_edges,i,4),rot_e_i_edges(ei_edges,i,4),ii_edges,num_int_e) for i in range(4)]

get orbits for the 4-th square of vertices

In [14]:
quad_vert_orb_rep = []
quad_vert_orb_fourth_cand = {}
for trip_orb in trip_vert_orb_rep:
    singletons = [[x] for x in set(trip_orb)]
    fourth_coord_orbits = the_graph.automorphism_group(partition=singletons+[list(set(the_graph.vertices()).difference(set(trip_orb)))],orbits=True,return_group=False)
    for orb in fourth_coord_orbits:
        quad_orb = trip_orb + (orb[0],)
        quad_vert_orb_rep += [quad_orb]
        quad_vert_orb_fourth_cand[quad_orb] = set(orb)
print(f"There are {len(quad_vert_orb_rep)} orbits in the quadruple product of vertices.")

There are 128 orbits in the quadruple product of vertices.


elements from nc partitions / TL diagrams

In [15]:
nc_partitions_4 = [[]] + [rot_partition([{0,1}],i,4) for i in range(4)] + [[{1,3}],[{0,2}]] + [rot_partition([{0,1,2}],i,4) for i in range(4)] + [[{0,1,2,3}],[{0,1},{2,3}],[{0,3},{1,2}]]

func_list_4_box_sp = [coeff_func_for_part(4,this_nc_part) for this_nc_part in nc_partitions_4]

elements with one internal edge

In [16]:
func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([],[(0,1)])

func_list_4_box_sp += [coeff_func_for_part(4,[],[(0,2)]),coeff_func_for_part(4,[],[(1,3)])]

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([{0,1}],[(0,3)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([{0,1}],[(0,2)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([{0,1}],[(2,3)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([{1,3}],[(0,1)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([{1,2,3}],[(0,1)])

func_list_4_box_sp += [coeff_func_for_part(4,[{0,1},{2,3}],[(1,2)]),coeff_func_for_part(4,[{0,3},{1,2}],[(0,1)])]

elements with two internal edges

In [17]:
func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([],[(0,1),(1,2)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([],[(0,1),(0,2)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([],[(0,1),(1,3)])

func_list_4_box_sp += [coeff_func_for_part(4,[],[(0,1),(2,3)]),coeff_func_for_part(4,[],[(0,3),(1,2)])]

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([{0,1}],[(0,3),(0,2)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([{0,1}],[(0,3),(2,3)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([{0,1}],[(0,2),(2,3)])

func_list_4_box_sp += [coeff_func_for_part(4,[{0,2}],[(1,2),(2,3)]),coeff_func_for_part(4,[{1,3}],[(0,1),(1,2)])]

elements with three internal edges

In [18]:
func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([],[(0,1),(0,3),(1,3)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([],[(0,3),(2,3),(1,2)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([],[(0,1),(0,2),(0,3)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([{0,1}],[(0,3),(0,2),(2,3)])

func_list_4_box_sp += [coeff_func_for_part(4,[],[(0,2),(0,3),(1,2)]),coeff_func_for_part(4,[],[(0,1),(1,3),(2,3)])]

func_list_4_box_sp += [coeff_func_for_part(4,[],[(1,3),(0,3),(1,2)]),coeff_func_for_part(4,[],[(0,1),(0,2),(2,3)])]

elements with four internal edges

In [19]:
func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([],[(0,3),(2,3),(1,2),(1,3)])

func_list_4_box_sp += coeff_funcs_parts_e_e_rotated_4([],[(0,3),(2,3),(1,2),(0,2)])

func_list_4_box_sp += [coeff_func_for_part(4,[],[(0,1),(1,2),(2,3),(0,3)]),coeff_func_for_part(4,[],[],[(0,0),(1,0),(2,0),(3,0)],[],1)]

elements with five internal edges

In [20]:
func_list_4_box_sp += [coeff_func_for_part(4,[],[(0,1),(1,2),(2,3),(0,3),(1,3)]),coeff_func_for_part(4,[],[(0,1),(1,2),(2,3),(0,3),(0,2)])]

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_rotated_4([],[(0,1)],[(0,0),(1,0),(2,0),(3,0)],1)

elements with six internal edges

In [21]:
func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_rotated_4([],[(0,1),(1,2)],[(0,0),(1,0),(2,0),(3,0)],1)

func_list_4_box_sp += [coeff_func_for_part(4,[],[(0,1),(2,3)],[(0,0),(1,0),(2,0),(3,0)],[],1),coeff_func_for_part(4,[],[(0,3),(1,2)],[(0,0),(1,0),(2,0),(3,0)],[],1)]

elements with seven internal edges

In [22]:
func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_rotated_4([],[(0,1),(1,2),(2,3)],[(0,0),(1,0),(2,0),(3,0)],1)

func_list_4_box_sp += [coeff_func_for_part(4,[],[],[(0,0),(0,1),(1,0),(2,0),(2,1),(3,1)],[(0,1)],2),coeff_func_for_part(4,[],[],[(0,0),(1,0),(1,1),(2,1),(3,0),(3,1)],[(0,1)],2)]

elements with eight internal edges

In [23]:
func_list_4_box_sp += [coeff_func_for_part(4,[],[(0,1),(1,2),(2,3),(3,0)],[(0,0),(1,0),(2,0),(3,0)],[],1)]

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1)],[(0,0),(0,1),(1,0),(2,0),(2,1),(3,1)],[(0,1)],2)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1)],[(0,0),(1,0),(1,1),(2,1),(3,0),(3,1)],[(0,1)],2)

elements with nine internal edges

In [24]:
func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[],[(0,0),(1,1),(2,1),(2,2),(3,0),(3,2)],[(0,1),(1,2),(0,2)],3)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1),(1,2)],[(0,0),(0,1),(1,0),(2,0),(2,1),(3,1)],[(0,1)],2)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1),(1,2)],[(0,0),(1,0),(1,1),(2,1),(3,0),(3,1)],[(0,1)],2)

func_list_4_box_sp += [coeff_func_for_part(4,[],[(0,1),(2,3)],[(0,0),(0,1),(1,0),(2,0),(2,1),(3,1)],[(0,1)],2),coeff_func_for_part(4,[],[(0,1),(2,3)],[(0,0),(1,0),(1,1),(2,1),(3,0),(3,1)],[(0,1)],2)]

func_list_4_box_sp += [coeff_func_for_part(4,[],[(1,2),(0,3)],[(0,0),(0,1),(1,0),(2,0),(2,1),(3,1)],[(0,1)],2),coeff_func_for_part(4,[],[(1,2),(0,3)],[(0,0),(1,0),(1,1),(2,1),(3,0),(3,1)],[(0,1)],2)]

elements with ten internal edges

In [25]:
func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1),(1,2),(2,3)],[(0,0),(0,1),(1,0),(2,0),(2,1),(3,1)],[(0,1)],2)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1),(1,2),(2,3)],[(0,0),(1,0),(1,1),(2,1),(3,0),(3,1)],[(0,1)],2)

func_list_4_box_sp += [coeff_func_for_part(4,[],[],[(0,0),(0,1),(0,2),(1,0),(2,0),(2,1),(2,2),(3,2)],[(0,1),(1,2)],3),coeff_func_for_part(4,[],[],[(0,0),(1,0),(1,1),(1,2),(2,2),(3,0),(3,1),(3,2)],[(0,1),(1,2)],3)]

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[],[(0,0),(1,0),(1,1),(1,2),(2,2),(3,0),(3,2)],[(0,1),(1,2),(0,2)],3)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[],[(0,0),(1,0),(1,1),(2,1),(2,2),(3,0),(3,2)],[(0,1),(1,2),(0,2)],3)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[],[(0,0),(0,1),(1,1),(2,1),(2,2),(3,0),(3,2)],[(0,1),(1,2),(0,2)],3)

elements with 11 internal edges

In [26]:
func_list_4_box_sp  += [coeff_func_for_part(4,[],[(0,1),(1,2),(2,3),(3,0)],[(0,0),(0,1),(1,0),(2,0),(2,1),(3,1)],[(0,1)],2),coeff_func_for_part(4,[],[(0,1),(1,2),(2,3),(3,0)],[(0,0),(1,0),(1,1),(2,1),(3,0),(3,1)],[(0,1)],2)]

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1)],[(0,0),(0,1),(0,2),(1,0),(2,0),(2,1),(2,2),(3,2)],[(0,1),(1,2)],3)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1)],[(0,0),(1,0),(1,1),(1,2),(2,2),(3,0),(3,1),(3,2)],[(0,1),(1,2)],3)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1)],[(0,0),(1,0),(1,1),(1,2),(2,2),(3,0),(3,2)],[(0,1),(1,2),(0,2)],3)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,3)],[(0,0),(1,0),(1,1),(1,2),(2,2),(3,0),(3,2)],[(0,1),(1,2),(0,2)],3)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(1,2)],[(0,0),(1,0),(1,1),(1,2),(2,2),(3,0),(3,2)],[(0,1),(1,2),(0,2)],3)

func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(2,3)],[(0,0),(1,0),(1,1),(1,2),(2,2),(3,0),(3,2)],[(0,1),(1,2),(0,2)],3)

elements with 12 internal edges

In [27]:
func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1)],[(0,0),(1,0),(1,1),(1,2),(2,2),(3,0),(3,2)],[(0,1),(1,2),(0,2)],3)

func_list_4_box_sp += [coeff_func_for_part(4,[],[],[(0,0),(0,3),(1,0),(1,1),(2,1),(2,2),(3,2),(3,3)],[(0,1),(1,2),(2,3),(3,0)],4)]

func_list_4_box_sp += [coeff_func_for_part(4,[],[],[(0,0),(0,1),(1,0),(1,2),(2,2),(2,3),(3,1),(3,3)],[(0,1),(0,2),(1,3),(2,3)],4)]

elements with 13 internal edges

In [28]:
func_list_4_box_sp += coeff_funcs_parts_e_e_e_i_i_i_rotated_4([],[(0,1)],[(0,0),(0,3),(1,0),(1,1),(2,1),(2,2),(3,2),(3,3)],[(0,1),(1,2),(2,3),(3,0)],4)

a few more odd ones with 4 internal vertices

In [29]:
func_list_4_box_sp += [coeff_func_for_part(4,[],[],[(0,1),(1,0),(1,2),(2,2),(2,3),(3,1),(3,3)],[(0,1),(0,2),(1,2),(1,3)],4)]

func_list_4_box_sp += [coeff_func_for_part(4, [], [], [(0, 0), (0, 1), (1, 0), (1, 2), (2, 2), (2, 3), (3, 1), (3, 3)], [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3)], 4)]

func_list_4_box_sp += [coeff_func_for_part(4, [], [(0, 1)], [(0, 0), (0, 1), (1, 0), (1, 2), (2, 2), (2, 3), (3, 1), (3, 3)], [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3)], 4)]

total number of elements

In [30]:
print(f"We have {len(func_list_4_box_sp)} diagram elements")

We have 204 diagram elements


Get the dimension of 4-box space spanned by the diagram elements

In [31]:
def myfunc3(x):
    return [func(x[0],x[1],x[2],x[3]) for func in func_list_4_box_sp]

if __name__ == '__main__':
    with mp.Pool() as p:
        Ap_array = p.map(myfunc3, quad_vert_orb_rep)

In [32]:
# want 128 for the McLaughlin
print(f"We now have rank {np.linalg.matrix_rank(np.array(Ap_array))}, spanned by {len(Ap_array[0])} elements")

We now have rank 128, spanned by 204 elements


In [33]:
raise Exception("Stop here")

Exception: Stop here

Find a linear combination of diagram elements that represents the flip

In [34]:
def myfunc1(x):
    """List values of above functions on a given tuple."""
    return [func(x[0], x[1], x[2], x[3]) for func in func_list_4_box_sp]

def func_4_perm(x, y, z, w):
    """Coefficient of the permutation element."""
    if x == z and y == w:
        return 1
    return 0

def myfunc2(x):
    return func_4_perm(x[0], x[1], x[2], x[3])

if __name__ == '__main__':
    with mp.Pool() as p:
        bp_array = p.map(myfunc2, quad_vert_orb_rep)

In [35]:
myvars = [sp.Symbol("x%d" % i) for i in range(len(func_list_4_box_sp))]
sols = sp.linsolve((sp.Matrix(Ap_array), sp.Matrix(bp_array)), myvars)

# get a concrete candidate of coefficients
sol_cand = list(sols)[0].subs([(thevar, 0) for thevar in myvars])

output = ""
for i in range(len(sol_cand)):
    if sol_cand[i] != 0:
        output += r' + \frac{%d}{%d} a_{%d}'\
            % (sol_cand[i].numerator, sol_cand[i].denominator, i)
print(output)

 + \frac{12865}{729} a_{0} + \frac{-1753}{729} a_{1} + \frac{-1753}{729} a_{2} + \frac{-1753}{729} a_{3} + \frac{-1753}{729} a_{4} + \frac{9301}{243} a_{5} + \frac{9301}{243} a_{6} + \frac{1760}{81} a_{7} + \frac{1760}{81} a_{8} + \frac{1760}{81} a_{9} + \frac{1760}{81} a_{10} + \frac{-37804}{243} a_{11} + \frac{-13657}{729} a_{12} + \frac{-13657}{729} a_{13} + \frac{-12457}{1458} a_{14} + \frac{-12457}{1458} a_{15} + \frac{-12457}{1458} a_{16} + \frac{-12457}{1458} a_{17} + \frac{-4016}{243} a_{18} + \frac{-4016}{243} a_{19} + \frac{79}{81} a_{20} + \frac{79}{81} a_{21} + \frac{79}{81} a_{22} + \frac{79}{81} a_{23} + \frac{271}{729} a_{24} + \frac{271}{729} a_{25} + \frac{271}{729} a_{26} + \frac{271}{729} a_{27} + \frac{1643}{1458} a_{28} + \frac{1643}{1458} a_{29} + \frac{1643}{1458} a_{30} + \frac{1643}{1458} a_{31} + \frac{-17728}{729} a_{32} + \frac{-17368}{729} a_{33} + \frac{-17368}{729} a_{34} + \frac{-17728}{729} a_{35} + \frac{-2230}{729} a_{36} + \frac{-50}{27} a_{37} + \fr

In [36]:
raise Exception("Stop here")

Exception: Stop here

## diagramatic presentation

In [37]:
mystr = ""
for j in range(21):
    mystr += r'%d--%d &' % (10*j, 10*j+9) + '\n'
    for i in range(10):
        k = 10 * j + i
        if k < len(func_list_4_box_sp):
            mystr += func_list_4_box_sp[k]("diag")
            if i < 9:
                mystr += '& \n'
    mystr += r'\\' + '\n'
print(mystr)

0--9 &
\begin{tikzpicture}
\draw[bgrect] (-1.3,-1.3) rectangle (1.3,1.3);
\node (corner0) at (-1,1) [ext-vert] {};
\node (corner1) at (1,1) [ext-vert] {};
\node (corner2) at (1,-1) [ext-vert] {};
\node (corner3) at (-1,-1) [ext-vert] {};
\end{tikzpicture}
& 
\begin{tikzpicture}
\draw[bgrect] (-1.3,-1.3) rectangle (1.3,1.3);
\node (corner0) at (-1,1) [ext-vert] {};
\node (corner1) at (1,1) [ext-vert] {};
\node (corner2) at (1,-1) [ext-vert] {};
\node (corner3) at (-1,-1) [ext-vert] {};
\draw [thick,double] (corner0) -- (corner1);
\end{tikzpicture}
& 
\begin{tikzpicture}
\draw[bgrect] (-1.3,-1.3) rectangle (1.3,1.3);
\node (corner0) at (-1,1) [ext-vert] {};
\node (corner1) at (1,1) [ext-vert] {};
\node (corner2) at (1,-1) [ext-vert] {};
\node (corner3) at (-1,-1) [ext-vert] {};
\draw [thick,double] (corner1) -- (corner2);
\end{tikzpicture}
& 
\begin{tikzpicture}
\draw[bgrect] (-1.3,-1.3) rectangle (1.3,1.3);
\node (corner0) at (-1,1) [ext-vert] {};
\node (corner1) at (1,1) [ext-vert] {};

In [ ]:
raise Exception("stop here.")